# PBupsModelmusc with Algorithmic Differentiation in Julia
4 more parameters.

In [2]:
# import packages..
import ForwardDiff
using ForwardDiff
# using PyPlot
import Base.convert
import Optim
using Optim

# === Upgrading from ForwardDiff v0.1 to v0.2
# instead of ForwardDiff.GradientNumber and ForwardDiff.HessianNumber, 
# we will use ForwardDiff.Dual

convert(::Type{Float64}, x::ForwardDiff.Dual) = Float64(x.value)
function convert(::Array{Float64}, x::Array{ForwardDiff.Dual}) 
    y = zeros(size(x)); 
    for i in 1:prod(size(x)) 
        y[i] = convert(Float64, x[i]) 
    end
    return y
end

immutable NumericPair{X,Y} <: Number
  x::X
  y::Y
end
Base.isless(a::NumericPair, b::NumericPair) = (a.x<b.x) || (a.x==b.x && a.y<b.y)


isless (generic function with 46 methods)

To evaluate how well a particular set of parameter values $\theta$ fits the behavioral data, we compute the probability of oberving the data given the model.

For each trial $i$, we will compute the likelihood of seeing the data under the model assuming that trials are independent. 

$P(D|\theta) = \prod_{i}P(d_i|t_{i,R},t_{i,L},\theta)$

$t_{i,R},t_{i,L}$ : the right and left click times on trial $i$

$d_i$ : the subject's decision on trial $i$

The best-fit parameter values are the parameters $\theta$ that maximize the likelihood (Maximum likelihood values)

To help maximize the likelihood(or log likelihood), we will compute the derivative $\partial P(d_i|t_{i,R},t_{i,L},\theta) / \partial\theta$ for each of the parameters in the set $\theta$.

After we get these gradients of 9 model parameters, we will apply them for optimization.

## Import data 

In [3]:
using MAT
ratdata = matread("testdata.mat")
# ratdata = matread("chrono_B069_rawdata.mat")

Dict{ByteString,Any} with 5 entries:
  "ratname"      => "B069"
  "daterange"    => 1x2 Array{Any,2}:…
  "avgdata"      => Dict{ASCIIString,Any}("is_probe"=>1x64537 Array{Bool,2}:…
  "rawdata"      => Dict{ASCIIString,Any}("is_probe"=>1x64537 Array{Any,2}:…
  "total_trials" => 64537.0

In [4]:
function trialdata(rawdata, trial::Int)
    if rawdata["pokedR"][trial] > 0
        rat_choice = 1;  # "R"
    else
        rat_choice = -1; # "L"
    end;
    
    if typeof(rawdata["rightbups"][trial]) <: Array
        rvec = vec(rawdata["rightbups"][trial])::Array{Float64,1};
    else
        rvec = Float64[]
    end
    if typeof(rawdata["leftbups"][trial]) <: Array
        lvec = vec(rawdata["leftbups"][trial])::Array{Float64,1};
    else
        lvec = Float64[]
    end
    
    return rvec, lvec, 
    rawdata["T"][trial]::Float64, rat_choice
end



trialdata (generic function with 1 method)

In [5]:
a = Float64[]

0-element Array{Float64,1}

In [6]:
@time RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], 1)

  0.069388 seconds (10.77 k allocations: 553.965 KB)


([0.0,0.001135000000000004,0.011165000000000001,0.01892,0.10638,0.20342,0.32964499999999997,0.33775999999999995],[0.0,0.19235,0.34360999999999997],0.35095899999999747,-1)

## bin_centers = make_bins(B, dx, binN)

In [7]:
"""
function bin_centers = make_bins(B, dx, binN)

Makes a series of points that will indicate bin centers. The first and
last points will indicate sticky bins. No "bin edges" are made-- the edge
between two bins is always implicity at the halfway point between their
corresponding centers. The center bin is always at x=0; bin spacing
(except for last and first bins) is always dx; and the position
of the first and last bins is chosen so that |B| lies exactly at the
midpoint between 1st (sticky) and 2nd (first real) bins, as well as
exactly at the midpoint between last but one (last real) and last
(sticky) bins.

Playing nice with ForwardDiff means that the *number* of bins must be predetermined.
So this function will not actually set the number of bins; what it'll do is determine their
locations. To accomplish this separation, the function uses as a third parameter binN,
which should be equal to the number of bins with bin centers > 0, as follows: 
   binN = ceil(B/dx)
and then the total number of bins will be 2*binN+1, with the center one always corresponding
to position zero. Use non-differentiable types for B and dx for this to work.
"""

function make_bins{T}(bins::Vector{T}, B, dx::T, binN)
    cnt = 1
    for i=-binN:binN
        bins[cnt] = i*dx
        cnt = cnt+1
    end
    
    if binN*dx == B
        bins[end] = B + dx
        bins[1] = -B - dx
    else
        bins[end] = 2*B - (binN-1)*dx
        bins[1] = -2*B + (binN-1)*dx
    end
end;

In [8]:
binN = ceil(4.1/0.25)
bins = zeros(typeof(binN), Int(binN*2+1))
@time make_bins(bins,4.1,0.25,binN)
bins

  0.092712 seconds (60.10 k allocations: 2.411 MB, 12.06% gc time)


35-element Array{Float64,1}:
 -4.2 
 -4.0 
 -3.75
 -3.5 
 -3.25
 -3.0 
 -2.75
 -2.5 
 -2.25
 -2.0 
 -1.75
 -1.5 
 -1.25
  ⋮   
  1.5 
  1.75
  2.0 
  2.25
  2.5 
  2.75
  3.0 
  3.25
  3.5 
  3.75
  4.0 
  4.2 

In [9]:
B = 4.1
dx_test = 0.25
binN = ceil(B/dx_test)
bins = zeros(typeof(binN), Int(binN*2+1))
@profile make_bins(bins,B,dx_test,binN)
Profile.print()
Profile.clear_malloc_data() 
bin_centers = bins
bins

1 task.jl; anonymous; line: 447
 1 ...4/IJulia/src/IJulia.jl; eventloop; line: 143
  1 ...src/execute_request.jl; execute_request_0x535c5df2; line: 183
   1 loading.jl; include_string; line: 282
    1 ...ia/lib/julia/sys.dylib; typeinf_ext; (unknown line)
     1 ...a/lib/julia/sys.dylib; typeinf; (unknown line)
      1 ...a/lib/julia/sys.dylib; typeinf_uncached; (unknown line)
       1 ...a/lib/julia/sys.dylib; abstract_eval; (unknown line)
        1 .../lib/julia/sys.dylib; abstract_eval_call; (unknown line)
         1 .../lib/julia/sys.dylib; abstract_call; (unknown line)
          1 ...lib/julia/sys.dylib; abstract_call_gf; (unknown line)
           1 ...lib/julia/sys.dylib; typeinf; (unknown line)
            1 ...lib/julia/sys.dylib; typeinf; (unknown line)
             1 ...ib/julia/sys.dylib; typeinf_uncached; (unknown line)
              1 ...ib/julia/sys.dylib; abstract_interpret; (unknown line)
               1 ...b/julia/sys.dylib; abstract_eval; (unknown line)
             

35-element Array{Float64,1}:
 -4.2 
 -4.0 
 -3.75
 -3.5 
 -3.25
 -3.0 
 -2.75
 -2.5 
 -2.25
 -2.0 
 -1.75
 -1.5 
 -1.25
  ⋮   
  1.5 
  1.75
  2.0 
  2.25
  2.5 
  2.75
  3.0 
  3.25
  3.5 
  3.75
  4.0 
  4.2 

## Global Variables

In [10]:
# Global variables 
const epsilon = 10.0^(-10);
const dx = 0.25;
const dt = 0.02;
const total_rate = 40;

## Parameters

a : decision variable, memory accumulator

$$ da =
  \begin{cases}
    0       & \quad \text{if, } |a| \geq B \\
    \sigma_adW + (\delta_{t,t_R} \cdot \eta C(t) - \delta_{t,t_L} \cdot \eta C(t))dt + \lambda adt  & \quad \text{otherwise, }\\
  \end{cases}
$$



The impact of each click (C) is affected by sensory adaptation that depends on clicks from both right and left sides:

$$ 
\frac{\mathrm d C}{\mathrm d t} = \frac{1-C}{\tau_\phi} + (1-\phi)C(\delta_{t,t_R}+\delta_{t,t_L}) 
$$


sigma2_a ($\sigma_a^2$) : a diffusion constant, parameterizing noise in a.

sigma2_s ($\sigma_s^2$) : parameterizing noise when adding evidence from a right or left pulse. (incoming sensory evidence)

sigma2_i ($\sigma_i^2$) : initial condition for the dynamical equation at $t=0$

lam ($\lambda$) : consistent drift in the memory a ($\lambda<0$ : leaky or forgetful case, $\lambda>0$ : unstable or impulsive case)

B : decision bound

bias : bias parameter determines the position of the threshold in a (which a Rightward decision is made)

phi ($\phi$) : parameterize sensory adaptation (by defining the dynamics of C ($\phi>1$ : Facilitation, $\phi<1$ : Depression, $\phi=1$ : absense of sensory adaptation)

tau_phi ($\tau_\phi$) :

lapse : The lapse rate parameterizes the probability of making a random response.

______
biased_sigma2_s :  
biased_input :  
biased_lapse :   


In [11]:
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   

sigma = params[1];
lam   = params[2];
c     = params[3];

## F = Fmatrix([sigma, lambda, c], bin_centers)

In [12]:
"""
function F = Fmatrix([sigma, lambda, c], bin_centers)

Uses globals
    dt
    dx
    epsilon       (=10.0^-10)

Returns a square Markov matrix of transition probabilities. 
Plays nice with ForwardDiff-- that is why bin_centers is a global vector (so that the rem
operations that go into defining the bins, which ForwardDiff doesn't know how to deal with,
stay outside of this differentiable function)

sigma  should be in (accumulator units) per (second^(1/2))
lambda should be in s^-1
c      should be in accumulator units per second
bin_centers should be a vector of the centers of all the bins. Edges will be at midpoints
       between the centers, and the first and last bin will be sticky.

dx is not used inside Fmatrix, because bin_centers specifies all we need to know.
dt *is* used inside Fmatrix, to convert sigma, lambda, and c into timestep units
"""
function Fmatrix{T}(F::AbstractArray{T,2},params::Vector, bin_centers)
    sigma2 = params[1];
    lam   = params[2];
    c     = params[3];
    
    sigma2_sbin = convert(Float64, sigma2)
      
    n_sbins = max(70, ceil(10*sqrt(sigma2_sbin)/dx))
    
    swidth = 5*sqrt(sigma2_sbin)
    sbinsize = swidth/n_sbins;#sbins[2] - sbins[1]
    base_sbins    = collect(-swidth:sbinsize:swidth)
    
    ps       = exp(-base_sbins.^2/(2*sigma2))
    ps       = ps/sum(ps);
    
    sbin_length = length(base_sbins)
    binN = length(bin_centers)

    mu = 0.
    for j in 2:binN
        if lam == 0
            mu = bin_centers[j]*exp(lam*dt)
        else
            mu = (bin_centers[j] + c/lam)*exp(lam*dt) - c/lam
        end
        
        for k in 1:sbin_length
            sbin = (k-1)*sbinsize + mu - swidth
             
            if sbin < bin_centers[1] #(bin_centers[1] + bin_centers[2])/2
                F[1,j] = F[1,j] + ps[k]
            elseif bin_centers[end] <= sbin#(bin_centers[end]+bin_centers[end-1])/2 <= sbins[k]
                F[end,j] = F[end,j] + ps[k]
            else # more condition
                if (sbin > bin_centers[1] && sbin < bin_centers[2])
                    lp = 1; hp = 2;
                elseif (sbin > bin_centers[end-1] && sbin < bin_centers[end])
                    lp = binN-1; hp = binN;
                else 
                    lp = floor(Int,((sbin-bin_centers[2])/dx) + 2)#find(bin_centers .<= sbins[k])[end]
                    hp = lp+1#Int(ceil((sbins[k]-bin_centers[2])/dx) + 1);
                end

                if lp < 1 
                    lp = 1; 
                end
                if hp < 1 
                    hp = 1;
                end

                if lp == hp
                    F[lp,j] = F[lp,j] + ps[k]
                else
                    dd = bin_centers[hp] - bin_centers[lp]
                    F[hp,j] = F[hp,j] + ps[k]*(sbin - bin_centers[lp])/dd
                    F[lp,j] = F[lp,j] + ps[k]*(bin_centers[hp] - sbin)/dd
                end                   
            end
        end
    end
    F[:,1] = 0; F[:,end] = 0; F[1,1] = 1; F[end,end] = 1;
end

Fmatrix (generic function with 1 method)

In [13]:
F = zeros(typeof(0.2),length(bin_centers),length(bin_centers))
@time Fmatrix(F,[0.2, 0., 0.0],bin_centers) # Fi
F

  0.370013 seconds (132.82 k allocations: 6.035 MB)


35x35 Array{Float64,2}:
 1.0  0.41182      0.218907     0.0917964    …  0.0          0.0          0.0
 0.0  0.197142     0.172131     0.112014        0.0          0.0          0.0
 0.0  0.187228     0.217923     0.187228        0.0          0.0          0.0
 0.0  0.119939     0.187228     0.217923        0.0          0.0          0.0
 0.0  0.0571377    0.119939     0.187228        0.0          0.0          0.0
 0.0  0.0202246    0.0571377    0.119939     …  0.0          0.0          0.0
 0.0  0.00531176   0.0202246    0.0571377       0.0          0.0          0.0
 0.0  0.00103206   0.00531176   0.0202246       0.0          0.0          0.0
 0.0  0.000148294  0.00103206   0.00531176      0.0          0.0          0.0
 0.0  1.59548e-5   0.000148294  0.00103206      0.0          0.0          0.0
 0.0  1.11105e-6   1.59548e-5   0.000148294  …  0.0          0.0          0.0
 0.0  0.0          1.11105e-6   1.59548e-5      0.0          0.0          0.0
 0.0  0.0          0.0          1.11105e

In [14]:
F = zeros(typeof(params[1]),length(bin_centers),length(bin_centers))
@time Fmatrix(F,params,bin_centers)
F

  0.000083 seconds (13 allocations: 7.547 KB)


35x35 Array{Float64,2}:
 1.0  0.461125    0.365007    0.274946    …  0.0         0.0         0.0
 0.0  0.0908452   0.0859592   0.0811359      0.0         0.0         0.0
 0.0  0.0945948   0.100835    0.0946575      0.0         0.0         0.0
 0.0  0.0888839   0.0945533   0.100892       0.0         0.0         0.0
 0.0  0.0737024   0.0889555   0.0945117      0.0         0.0         0.0
 0.0  0.0612142   0.0736883   0.0890271   …  0.0         0.0         0.0
 0.0  0.044833    0.0612781   0.0736743      0.0         0.0         0.0
 0.0  0.0329193   0.0448354   0.0613421      0.0         0.0         0.0
 0.0  0.0212918   0.0329616   0.0448377      0.0         0.0         0.0
 0.0  0.0138235   0.021298    0.0330038      0.0         0.0         0.0
 0.0  0.0078944   0.0138445   0.0213041   …  0.0         0.0         0.0
 0.0  0.00453257  0.0078985   0.0138655      0.0         0.0         0.0
 0.0  0.00228512  0.00454054  0.00790261     3.13941e-7  0.0         0.0
 ⋮                         

## logProbRight 
### (params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)

* params = [sigma_a, sigma_s, sigma_i, lambda, B, bias, phi, tau_phi, lapse]
* RightClickTimes vector with elements indicating times of right clicks
* LeftClickTimes vector with elements indicating times of left clicks
* Nsteps number of timesteps to simulate 

a (column vector representing distribution of values of accumulator a)

a_trace (length(bin_centers)-by-Nsteps+1), a trace of the distribution of a as 
    a function of time
    
c_trace (row vector Nsteps+1 long, effective value of c as 
    a function of time after adaptation)


In [18]:
"""
version with inter-click interval(ici) for c_eff_net / c_eff_tot (followed the matlab code) 
(which was using dt for c_eff)

function logProbRight(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)

    Nsteps            number of timesteps to simulate
    RightClickTimes   vector with elements indicating times of right clicks
    LeftClickTimes    vector with elements indicating times of left clicks

    a      (column vector representing distribution of values of accumulator a)

    a_trace (length(bin_centers)-by-Nsteps+1), a trace of the distribution of a as 
            a function of time
    c_trace (row vector Nsteps+1 long, effective value of c as 
            a function of time after adaptation)

Takes params
    sigma_a = params[1]; sigma_s = params[2]; sigma_i = params[3]; 
    lambda = params[4]; B = params[5]; bias = params[6]; 
    phi = params[7]; tau_phi = params[8]; lapse = params[9]

Returns the log of the probability that the agent chose Right. 
"""

function logProbRight(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int)
    sigma_a = params[1]; sigma_s = params[2]; sigma_i = params[3]; 
    lambda = params[4]; B = params[5]; bias = params[6]; 
    phi = params[7]; tau_phi = params[8]; lapse = params[9]
    biased_sigma2_s = params[10]; biased_input = params[11]; biased_lapse = params[12];
    
    if isempty(RightClickTimes) RightClickTimes = zeros(0) end;
    if isempty(LeftClickTimes ) LeftClickTimes  = zeros(0) end;
    
    NClicks = zeros(Int, Nsteps); 
    Lhere  = zeros(Int, length(LeftClickTimes)); 
    Rhere = zeros(Int, length(RightClickTimes)); 
    
    for i in 1:length(LeftClickTimes)
        Lhere[i] = ceil((LeftClickTimes[i]+epsilon)/dt)
    end
    for i in 1:length(RightClickTimes)
        Rhere[i] = ceil((RightClickTimes[i]+epsilon)/dt)
    end    
    
    for i in Lhere  
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end
    for i in Rhere  
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end
    
    # === Upgrading from ForwardDiff v0.1 to v0.2
    # instead of using convert we can use floor(Int, ForwardDiff.Dual) and
    # ceil(Int, ForwardDiff.Dual)

    binN = ceil(Int, B/dx)#Int(ceil(my_B/dx))  
    binBias = floor(Int, bias/dx) + binN+1  
    bin_centers = zeros(typeof(dx), binN*2+1)
    make_bins(bin_centers, B, dx, binN)  

    ## biased params added
    a0 = zeros(typeof(sigma_a),length(bin_centers))
    a0[binN+1] = 1-lapse/2-biased_lapse/2; a0[1] = biased_lapse/2; a0[end] = lapse/2;
    
    temp_l = [NumericPair(LeftClickTimes[i],-1) for i=1:length(LeftClickTimes)]
    temp_r = [NumericPair(RightClickTimes[i],1) for i=1:length(RightClickTimes)]
    allbups = sort!([temp_l; temp_r])

    c_eff = 0.
    cnt = 0
        
    Fi = zeros(typeof(sigma_i),length(bin_centers),length(bin_centers))
    Fmatrix(Fi,[sigma_i, 0., 0.0], bin_centers)
    a = Fi*a0;
    
    F0 = zeros(typeof(sigma_a),length(bin_centers),length(bin_centers))
    Fmatrix(F0,[sigma_a*dt, lambda, 0.0], bin_centers)
    for i in 2:Nsteps 
        c_eff_tot = 0.
        c_eff_net = 0.
        net_c_input = 0.
        net_i_input = 0.
        
        if NClicks[i-1]==0
            c_eff_tot = 0.
            c_eff_net = 0.
            a = F0*a
        else
            for j in 1:NClicks[i-1]
                if cnt != 0 || j != 1
                    ici = allbups[cnt+j].x - allbups[cnt+j-1].x
                    c_eff = 1 + (c_eff*phi - 1)*exp(-ici/tau_phi)
                    c_eff_tot = c_eff_tot + c_eff
                    c_eff_net = c_eff_net + c_eff*allbups[cnt+j].y
                    
                    ## biased params added
                    net_c_input = (c_eff_tot+c_eff_net)/2
                    net_i_input = c_eff_tot-c_eff_net
                    
                    c_eff_net = net_c_input*biased_input - net_i_input
                end
                if j == NClicks[i-1]
                    cnt = cnt+j
                end
            end
            
            gain_first = 0
            ## biased params added
            if gain_first == 1
                net_sigma = sigma_a*dt + (sigma_s*net_i_input)/total_rate + net_c_input*biased_input*biased_sigma2_s/total_rate
            else
                net_sigma = sigma_a*dt + (sigma_s*net_i_input)/total_rate + net_c_input*biased_sigma2_s/total_rate
            end
            F = zeros(typeof(net_sigma),length(bin_centers),length(bin_centers))
            Fmatrix(F,[net_sigma, lambda, c_eff_net/dt], bin_centers)
            a = F*a
        end
    end
#     plot(1:Nsteps+1,c_trace[:])    
#     imshow(a_trace, interpolation="none")
    pright = sum(a[binBias+2:end]) + 
    a[binBias]*((bin_centers[binBias+1] - bias)/dx/2) +
    a[binBias+1]*(0.5 + (bin_centers[binBias+1] - bias)/dx/2)
    
    return log(pright)
end

function logLike(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int, rat_choice::Int)
    if rat_choice > 0
        # println("Right")
        return logProbRight(params, RightClickTimes, LeftClickTimes, Nsteps)
    elseif rat_choice < 0
        # println("Left")
        return log(1 - exp(logProbRight(params, RightClickTimes, LeftClickTimes, Nsteps)))
    end
end

logLike (generic function with 1 method)

## single_trial
### (params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)

In [19]:
""" 
function (LL, LLgrad) = 
    single_trial(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)

Computes the log likelihood according to Bing's model, and returns log likelihood, gradient

params is a vector whose elements, in order, are
    sigma_a    square root of accumulator variance per unit time sqrt(click units^2 per second)
    sigma_s    standard deviation introduced with each click (will get scaled by click adaptation)
    sigma_i    square root of initial accumulator variance sqrt(click units^2)
    lambda     1/accumulator time constant (sec^-1). Positive means unstable, neg means stable
    B          sticky bound height (click units)
    bias       where the decision boundary lies (click units)
    phi        click adaptation/facilitation multiplication parameter
    tau_phi    time constant for recovery from click adaptation (sec)
    lapse      2*lapse fraction of trials are decided randomly

rat_choice     should be either "R" or "L"


RETURNS:


"""
# === Upgrading from ForwardDiff v0.1 to v0.2
# for Retrieving Lower-Order Results
#     # old way
#     answer, results = ForwardDiff.hessian(f, x, AllResults)
#     v = ForwardDiff.value(results)
#     g = ForwardDiff.gradient(results)
#     h = ForwardDiff.hessian(results) # == answer

#     # new way
#     out = HessianResult(x)
#     ForwardDiff.hessian!(out, f, x)
#     v = ForwardDiff.value(out)
#     g = ForwardDiff.gradient(out)
#     h = ForwardDiff.hessian(out)

function single_trial(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int, rat_choice::Int)
    function llikey(params::Vector)
        logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end

    result =  GradientResult(params)
    
    ForwardDiff.gradient!(result, llikey, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
   
    return LL, LLgrad
end

# function single_trial(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)
#     function llikey(params::Vector)
#         logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
#     end

#     result =  HessianResult(params) 
#     ForwardDiff.hessian!(result, llikey, params);
    
#     LL     = ForwardDiff.value(result)
#     LLgrad = ForwardDiff.gradient(result)
#     LLhessian = ForwardDiff.hessian(result)
   
#     return LL, LLgrad, LLhessian
# end

single_trial (generic function with 1 method)

In [20]:
### =============== testing 1 ================= ####

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   

RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], 1)
Nsteps = Int(cld(maxT,dt))

@time logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
### =========================================== #### 

  0.213587 seconds (122.32 k allocations: 6.712 MB)


-0.061441542554305784

In [21]:
### =============== testing 2 ================= ####
@time LL, LLgrad = single_trial(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
println(LL)
println(LLgrad)
### =========================================== ####

  1.952016 seconds (1.16 M allocations: 53.366 MB, 1.43% gc time)
-0.061441542554305784
[-0.013924431739652634,-0.00212980836960904,-0.04088267955323025,-0.005989831856647342,-0.0,0.028179747614771773,0.014758076936307426,-0.30603372348125085,-0.5260175463079088,-0.00175993441792658,-0.08123661325677888,0.005666620149731098]


In [22]:
LL

-0.061441542554305784

In [23]:
LLgrad

12-element Array{Float64,1}:
 -0.0139244 
 -0.00212981
 -0.0408827 
 -0.00598983
 -0.0       
  0.0281797 
  0.0147581 
 -0.306034  
 -0.526018  
 -0.00175993
 -0.0812366 
  0.00566662

# Maximize LL over parameter space
### Optimization with Optim.jl


In [24]:
function ComputeLL(params::Vector, ratdata, ntrials::Int)
    LL        = 0.
        
    for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))

        LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
        LL        = LL + LLi;
    end
    
    LL = -LL
    return LL
end

ComputeLL (generic function with 1 method)

In [25]:
function ComputeGrad(params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
            Nsteps = Int(ceil(maxT/dt))

            LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
            LL        = LL + LLi;
        end

        LL = -LL
        return LL
    end

    result =  GradientResult(params)
    
    ForwardDiff.gradient!(result, WrapperLL, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
   
    return LL, LLgrad
end

ComputeGrad (generic function with 1 method)

In [26]:
@profile ComputeLL(params,ratdata["rawdata"],300)
Profile.print()
Profile.clear_malloc_data() 

2   ...ia/lib/julia/sys.dylib; trunc; (unknown line)
366 task.jl; anonymous; line: 447
 366 .../IJulia/src/IJulia.jl; eventloop; line: 143
  366 ...rc/execute_request.jl; execute_request_0x535c5df2; line: 183
   366 loading.jl; include_string; line: 282
    1   ...a/lib/julia/sys.dylib; typeinf_ext; (unknown line)
     1 ...a/lib/julia/sys.dylib; typeinf; (unknown line)
      1 ...a/lib/julia/sys.dylib; typeinf_uncached; (unknown line)
       1 ...a/lib/julia/sys.dylib; abstract_eval; (unknown line)
        1 .../lib/julia/sys.dylib; abstract_eval_call; (unknown line)
         1 .../lib/julia/sys.dylib; abstract_call; (unknown line)
          1 ...lib/julia/sys.dylib; abstract_call_gf; (unknown line)
           1 ...lib/julia/sys.dylib; typeinf; (unknown line)
            1 ...lib/julia/sys.dylib; typeinf; (unknown line)
             1 ...ib/julia/sys.dylib; typeinf_uncached; (unknown line)
              1 ...ib/julia/sys.dylib; abstract_interpret; (unknown line)
               1 ...b/

In [26]:
@time ComputeGrad(params,ratdata["rawdata"],300)

  1.357869 seconds (212.11 k allocations: 436.365 MB, 4.02% gc time)


(186.18934811722343,[-7.921493720489457,-1.903502701632662,-27.457935987828012,-1.9355101592619097,-0.0,-1.993316686050263,41.37803503344536,-274.5018818728638,-276.7710505950882])

In [28]:
function ComputeHess(params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], i)
            Nsteps = Int(ceil(maxT/dt))

            LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
            LL        = LL + LLi;
        end

        LL = -LL
        return LL
    end

    result =  HessianResult(params)
    
    ForwardDiff.hessian!(result, WrapperLL, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
    LLHess = ForwardDiff.hessian(result)
   
    return LL, LLgrad, LLHess
end

ComputeHess (generic function with 1 method)

In [29]:
@time ComputeHess(params,ratdata,27)

 24.271064 seconds (3.45 M allocations: 822.294 MB, 0.71% gc time)


(36.78720340423782,[-1.4552997840241386,-0.196446839583693,-5.161497619638472,-0.013893477788013186,-0.0,6.108956139555055,6.046100039814632,-49.92610931987217,-97.81308752135685,-0.24060094113327068,-16.833533942557096,1.7297261901620709],
12x12 Array{Float64,2}:
   0.268743     0.00930384    0.858212   …   -1.84333    0.45802   
   0.00930384  -0.00188868    0.0353593      -0.289048   0.0686926 
   0.858212     0.0353593     4.5069         -4.94005    1.52369   
   0.0953173    2.21095e-5   -1.2             0.197307   0.00744752
  -0.0         -0.0          -0.0            -0.0       -0.0       
   0.474159     0.150017      0.0926605  …   11.7323    -1.25247   
   1.55022      0.0980763     5.29229        -0.516745  -1.9281    
 -12.2887      -0.840865    -40.4747         22.4132    15.4206    
   9.80667      1.46332      34.4316         96.8118    -6.19778   
   0.0529454    0.00382848    0.184684       -0.314211   0.0728537 
  -1.84333     -0.289048     -4.94005    …  -33.8779   

## Parallel Computing

In [ ]:
n_core = 4
if nworkers() < n_core
    addprocs(n_core-nworkers(); exeflags="--check-bounds=yes")
end
@assert nprocs() > n_core
@assert nworkers() >= n_core



In [ ]:
a = SharedArray(Float64,10)
@sync @parallel for i=1:10
  a[i] = i
end
a

In [ ]:
# addprocs(3)

# a = SharedArray(Float64,10)
# @parallel for i=1:10
#   a[i] = i
# end

function ComputeLL_par(LLs::SharedArray{Float64,1}, params::Vector, ratdata, ntrials::Int)
    @sync @parallel for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))
        LLs[i] = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end

    LL = sum(LLs)
    
    LL = -LL
    return LL
end

function ComputeGrad_par(LLs::SharedArray{Float64,1}, params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        @sync @parallel for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
            Nsteps = Int(ceil(maxT/dt))

            LLs[i] = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
        end
        
        LL = sum(LLs)
        LL = -LL
        return LL
    end

    result =  GradientResult(params)
    
    ForwardDiff.gradient!(result, WrapperLL, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
   
    return LL, LLgrad
end


In [ ]:
LLs = SharedArray(Float64, 10)
@time SumLikey_LL_par(LLs,params, ratdata, 10)

In [ ]:
@profile SumLikey_LL_par(params, ratdata, 27) # sum of LL for trial (1-27)
Profile.print()
Profile.clear_malloc_data() 

In [ ]:
@time SumLikey_par(params, ratdata, 27)

In [ ]:
function Likely_all_trials{T}(LL::AbstractArray{T,1},params::Vector, ratdata, ntrials::Int)     
    for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))

        LL[i] = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end
end

In [ ]:
likely_all = zeros(ntrials)
@time Likely_all_trials(likely_all, params, ratdata["rawdata"], 27)
likely_all

In [ ]:
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]

ntrials = 27

function LL_f(params::Vector)
    return ComputeLL(params, ratdata["rawdata"], ntrials)
end

function LL_g!(params::Vector, grads::Vector)
#     LL, LLgrad, LLhess = llikey(params)
    LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
    for i=1:length(params)
        grads[i] = LLgrad[i]
    end
end

function LL_fg!(params::Vector, grads)
    LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
    for i=1:length(params)
        grads[i] = LLgrad[i]
    end
    return LL
end

d4 = DifferentiableFunction(LL_f,
                            LL_g!,
                            LL_fg!)

In [ ]:
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 6.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]

l = [0, 0, 0, -5, 5, -5, 0.01, 0.005, 0]
u = [200, 200, 30, +5, 25, +5, 1.2, 0.7, 1]

res = optimize(d4, params, l, u, Fminbox(); 
         optimizer = GradientDescent, optimizer_o = OptimizationOptions(g_tol = 1e-12,
                                                                        x_tol = 1e-10,
                                                                        f_tol = 1e-6,                                                                        
                                                                        iterations = 200,
                                                                        store_trace = true,
                                                                        show_trace = true,
                                                                        extended_trace = true))


In [ ]:
x_bf = res.f_minimum

In [ ]:
res

In [ ]:
function main()
    
    ratname = "B069"
    # data import
    ratdata = matread(*("chrono_",ratname,"_rawdata.mat"))
    println("rawdata of ", ratname, " imported" )

    # number of trials
    ntrials = Int(ratdata["total_trials"])

    # Parameters
    sigma_a = 1.; sigma_s = 0.1; sigma_i = 0.2; 
    sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
    lam = -0.0005; B = 6.1; bias = 0.1; 
    phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
    params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse]

    l = [0., 0., 0., -5., 5., -5., 0.01, 0.005, 0.]
    u = [200., 200., 30., 5., 25., 5., 1.2, 0.7, 1.]

    # @code_warntype SumLikey(params, ratdata, ntrials)

    function LL_f(params::Vector)
        return SumLikey_LL(params, ratdata, ntrials)
    end

    function LL_g!(params::Vector, grads::Vector)
        LL, LLgrad = SumLikey(params, ratdata, ntrials)
        for i=1:length(params)
            grads[i] = LLgrad[i]
        end
    end

    function LL_fg!(params::Vector, grads)
        LL, LLgrad = SumLikey(params, ratdata, ntrials)
        for i=1:length(params)
            grads[i] = LLgrad[i]
        end
        return LL
    end

    d4 = DifferentiableFunction(LL_f,
                                LL_g!,
                                LL_fg!)

    res = optimize(d4, params, l, u, Fminbox(); 
             optimizer = GradientDescent, optimizer_o = OptimizationOptions(g_tol = 1e-12,
                                                                            iterations = 200,
                                                                            store_trace = true,
                                                                            show_trace = true))

end

In [ ]:
main()

In [ ]:
y = d4.f(params)

In [ ]:
grads = Array(Float64, length(params))
grads
# LL_g!(params, grads)
d4.g!(params, grads)
grads

In [ ]:
grads = Array(Float64, length(params))
d4.g!(params, grads)
grads

In [ ]:
y = d4.fg!(params, grads)

In [ ]:
optimize(d4, params, GradientDescent(),
               OptimizationOptions(g_tol = 1e-12,
                                   iterations = 200,
                                   store_trace = true,
                                   show_trace = true,
                                   extended_trace = true))

# For Performance improvement

## 1. 

In [ ]:
function xinc(x)
    return [x, x+1, x+2]
end

function loopinc()
    y = 0
    for i = 1:10^7
        ret = xinc(i)
        y += ret[2]
    end
    y
end

In [ ]:
function xinc!{T}(ret::AbstractVector{T}, x::T)
    ret[1] = x
    ret[2] = x+1
    ret[3] = x+2
    nothing
end

function loopinc_prealloc()
    ret = Array(Int, 3)
    y = 0
    for i = 1:10^7
        xinc!(ret, i)
        y += ret[2]
    end
    y
end

In [ ]:
@time loopinc()
@time loopinc_prealloc()

## 2.

In [ ]:
function f()
    n = 1000000
    a = rand(n)
    x = rand(n)


    function g(x, y)
        x+y
    end

    @time for i in 1:n
        cl(x) = g(x, a[i])
        cl(x[i])
    end

    a_val = 0.0
    cl2(x) = g(x, a_val)
    @time for i in 1:n
        a_val = a[i]
        cl2(x[i])
    end

    @time for i in 1:n
      g(x[i], a[i])
    end
end
f()

## 3. 

In [ ]:
function calcSum{T}(x::Vector{T}, y::Vector{T}, Ei::T, Ef::T)
    mysum=0.0::Float64;
    for(i in 1:N)
         if( Ei < x[i] <= Ef)
             mysum += y[i];
         end
    end
    return(mysum);
end

In [ ]:
function calcSum_simd{T}(x::Vector{T}, y::Vector{T}, Ei::T, Ef::T)
    mysum = zero(T)
    @inbounds @simd for i in eachindex(x, y)
         mysum += ifelse(Ei < x[i] <= Ef, y[i], zero(T)) 
         
    end
    return mysum
end

In [ ]:
N = 10000000
y = rand(N);
x = rand(N)
Ei = 0.2;
Ef = 0.7;

@time calcSum_simd(x,y,Ei, Ef);
@time calcSum(x,y,Ei, Ef)

In [ ]:
sigma2_sbin = 0.01
n_sbins = 70
swidth = 5*sqrt(sigma2_sbin)
sbinsize = swidth/n_sbins;#sbins[2] - sbins[1]

In [ ]:
function f_test(RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)
    if isempty(RightClickTimes) RightClickTimes = zeros(0) end;
    if isempty(LeftClickTimes ) LeftClickTimes  = zeros(0) end;
    
    LeftClicks  = zeros(Int, 1, Nsteps); 
    RightClicks = zeros(Int, 1, Nsteps); 
    
    for i in ceil((LeftClickTimes+epsilon)/dt)  LeftClicks[Int(i)]  = LeftClicks[Int(i)] + 1 end
    for i in ceil((RightClickTimes+epsilon)/dt) RightClicks[Int(i)] = RightClicks[Int(i)] + 1 end
end

In [ ]:
function f_test2(RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)
    if isempty(RightClickTimes) RightClickTimes = zeros(0) end;
    if isempty(LeftClickTimes ) LeftClickTimes  = zeros(0) end;
    
    LeftClicks  = zeros(Int, Nsteps); 
    RightClicks = zeros(Int, Nsteps); 
    NClicks = zeros(Int, Nsteps); 
    Lhere  = zeros(Int, length(LeftClickTimes)); 
    Rhere = zeros(Int, length(RightClickTimes)); 
    
    for i in 1:length(LeftClickTimes)
        Lhere[i] = ceil((LeftClickTimes[i]+epsilon)/dt)
    end
    for i in 1:length(RightClickTimes)
        Rhere[i] = ceil((RightClickTimes[i]+epsilon)/dt)
    end    
    
    for i in Lhere  
        LeftClicks[Int(i)]  = LeftClicks[Int(i)]  + 1 
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end
    for i in Rhere  
        RightClicks[Int(i)] = RightClicks[Int(i)] + 1 
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end
    
    temp_l = [NumericPair(LeftClickTimes[i],-1) for i=1:length(LeftClickTimes)]
    temp_r = [NumericPair(RightClickTimes[i],1) for i=1:length(RightClickTimes)]
    allbups = sort!([temp_l; temp_r])

    cnt_r = 0
    
    adapted = zeros(length(allbups))
    
    for i in 2:length(allbups)
        ici = allbups[i].x - allbups[i-1].x
        if ici == 0 
            adapted[i-1] = 0
            adapted[i] = 0
        else
            adapted[i] = 1 + (adapted[i-1]*phi - 1)*exp(-ici/tau_phi)
        end
    end
        
    Fi = zeros(typeof(sigma_i),length(bin_centers),length(bin_centers))
    Fmatrix(Fi,[sigma_i, 0, 0.0], bin_centers)
    a = Fi*a0;
    
    F0 = zeros(typeof(sigma_a),length(bin_centers),length(bin_centers))
    Fmatrix(F0,[sigma_a*dt, lambda, 0.0], bin_centers)
    for i in 2:Nsteps 
        c_eff_tot = 0
        c_eff_net = 0
        if NClicks[i-1]==0#(RightClicks[i-1]==0) && (LeftClicks[i-1]==0)
            c_eff_tot = 0
            c_eff_net = 0

            a = F0*a
        else
            for j in 1:NClicks[i-1]
                if cnt_r != 0 || j != 1
                    c_eff_tot = c_eff_tot + adapted[cnt_r+j]
                    c_eff_net = c_eff_net + adapted[cnt_r+j]*allbups[cnt_r+j].y
                end
                if j == NClicks[i-1]
                    cnt_r = cnt_r+j
                end
            end

            net_sigma = sigma_a*dt + (sigma_s*c_eff_tot)/total_rate
            F = zeros(typeof(net_sigma),length(bin_centers),length(bin_centers))
            Fmatrix(F,[net_sigma, lambda, c_eff_net/dt], bin_centers)
            a = F*a
        end
    end;   
end

In [ ]:
@time f_test2(RightClickTimes, LeftClickTimes,Nsteps)

In [ ]:
t = f_test2(RightClickTimes, LeftClickTimes,Nsteps)
t

In [ ]:
# t[1].x
length(t)

In [ ]:
immutable NumericPair{X,Y} <: Number
  x::X
  y::Y
end
Base.isless(a::NumericPair, b::NumericPair) = (a.x<b.x) || (a.x==b.x && a.y<b.y)

In [ ]:
function sort_numeric_pair()
  temp = rand(500)
  temp2 = [NumericPair(temp[i],i) for i=1:length(temp)]
  @time sort!(temp2)
end

In [ ]:
sort_numeric_pair()

In [ ]:
function rosenbrock(x)
   a = one(eltype(x))
   b = 100 * a
   result = zero(eltype(x))
   for i in 1:length(x)-1
       result += (a - x[i])^2 + b*(x[i+1] - x[i]^2)^2
   end
   return result
end

In [ ]:
x = rand(100);

In [ ]:
@time ForwardDiff.gradient(rosenbrock, x, Chunk{1}());

In [ ]:
@time ForwardDiff.gradient(rosenbrock, x, Chunk{4}());

In [ ]:
@time ForwardDiff.gradient(rosenbrock, x, Chunk{10}());

In [ ]:
@time ForwardDiff.gradient(rosenbrock, x, Chunk{16}());

In [ ]:
@time ForwardDiff.gradient(rosenbrock, x);

In [ ]:
out = GradientResult(x)

In [ ]:
@time ForwardDiff.gradient!(out, rosenbrock, x, Chunk{10}())

In [ ]:
out = HessianResult(x)
@time ForwardDiff.hessian!(out, rosenbrock, x)
hess = ForwardDiff.hessian(out)

In [ ]:
imshow(log(abs(hess)),interpolation = "none")

In [ ]:
@time ForwardDiff.hessian(rosenbrock, x)

## Parallel Process

In [ ]:
@everywhere function count_heads(n)
    c::Int = 0
    for i=1:n
        c += rand(Bool)
    end
    c
end

In [ ]:
a = @spawn count_heads(100000000)
b = @spawn count_heads(100000000)
fetch(a)+fetch(b)

In [ ]:
nheads = @parallel (+) for i=1:200000000
  Int(rand(Bool))
end

In [ ]:
# procs()
nprocs()
# addprocs(4)
procs()

In [ ]:
nheads = @parallel (+) for i=1:200000000
  Int(rand(Bool))
end

In [ ]:
@everywhere function tt(n)
    s::Int = 0
    for i=1:n
        s += i
    end
    s
end

function t(n)
    s::Int = 0
    for i=1:n
        s += i
    end
    s
end

In [ ]:
n = 1000
@time t(n)

In [ ]:
a = @spawn tt(n)
b = @spawn tt(n)
c = @spawn tt(n)
d = @spawn tt(n)
fetch(a)+fetch(b)+fetch(c)+fetch(d)

In [ ]:
a = zeros(100000)
@parallel for i=1:100000
  a[i] = i
end
# However, this code will not initialize all of a, 
# since each process will have a separate copy of it. 
# Parallel for loops like these must be avoided. 
# Fortunately, distributed arrays can be used to get around this limitation

In [ ]:
# Using “outside” variables in parallel loops 
# is perfectly reasonable if the variables are read-only:

@everywhere function f(a)
    a+1
end
a = randn(1000)
@parallel (+) for i=1:1000
  f(a[rand(1:end)])
end

In [ ]:
dzeros((100,100), workers()[1:4], [1,4])

In [ ]:
[i+j for i = 1:5, j = 1:5]

In [ ]:
@DArray [i+j for i = 1:5, j = 1:5]

In [ ]:
darray_closeall()

In [ ]:
workers()
# rmprocs(12:15)

In [ ]:
n_core = 4
if nworkers() < n_core
    addprocs(n_core-nworkers(); exeflags="--check-bounds=yes")
end
@assert nprocs() > n_core
@assert nworkers() >= n_core

@everywhere using DistributedArrays

In [ ]:
a = dzeros((100001,1), workers()[1:n_core], [n_core,1])

In [ ]:
function par_assign(d::DArray)
    DArray(size(d),procs(d)) do I
        res = zeros(Int,length(I[1]))        
        for i = 1:length(I[1])
            res[i] = I[1][i]
        end
        res
    end
end

# a = zeros(100000)
# @parallel for i=1:100000
#   a[i] = i
# end


In [ ]:
a = dzeros((100000,1), workers()[1:n_core], [n_core,1])
@profile par_assign(a)

In [ ]:
a = SharedArray(Float64,10)
@parallel for i=1:10
  a[i] = i
end

In [ ]:
a

In [ ]:
Profile.print()
Profile.clear_malloc_data() 

In [ ]:
function assign()
    a = zeros(100000)
    for i=1:100000
      a[i] = i
    end
    a
end

In [ ]:
@time assign

In [ ]:
@everywhere function life_step(d::DArray)
    DArray(size(d),procs(d)) do I
        top   = mod(first(I[1])-2,size(d,1))+1
        bot   = mod( last(I[1])  ,size(d,1))+1
        left  = mod(first(I[2])-2,size(d,2))+1
        right = mod( last(I[2])  ,size(d,2))+1

        old = Array(Bool, length(I[1])+2, length(I[2])+2)
        old[1      , 1      ] = d[top , left]   # left side
        old[2:end-1, 1      ] = d[I[1], left]
        old[end    , 1      ] = d[bot , left]
        old[1      , 2:end-1] = d[top , I[2]]
        old[2:end-1, 2:end-1] = d[I[1], I[2]]   # middle
        old[end    , 2:end-1] = d[bot , I[2]]
        old[1      , end    ] = d[top , right]  # right side
        old[2:end-1, end    ] = d[I[1], right]
        old[end    , end    ] = d[bot , right]

        life_rule(old)
    end
end

In [ ]:
@everywhere function life_rule(old)
    m, n = size(old)
    new = similar(old, m-2, n-2)
    for j = 2:n-1
        for i = 2:m-1
            nc = +(old[i-1,j-1], old[i-1,j], old[i-1,j+1],
                   old[i  ,j-1],             old[i  ,j+1],
                   old[i+1,j-1], old[i+1,j], old[i+1,j+1])
            new[i-1,j-1] = (nc == 3 || nc == 2 && old[i,j])
        end
    end
    new
end

In [ ]:
test1 = floor(drand((100,100), workers()[1:n_core], [2,2])*2)
# test1 = dzeros((100,100), workers()[1:n_core], [2,2])

In [ ]:
life_step(test1)

In [ ]:
M = Matrix{Float64}[rand(1000,1000) for i=1:10]
pmap(svd, M)

In [ ]:
sigma2_sbin = 37
ceil(10*sqrt(sigma2_sbin)/dx)

In [ ]:
binN = ceil(4.1/0.25)
bins = zeros(typeof(binN), Int(binN*2+1))
@time make_bins(bins,4.1,0.25,binN)
bins
bin_centers = bins

In [ ]:
sigma2 = 1
lam   = 5
c     = 20

n_sbins = max(70, ceil(10*sqrt(sigma2)/dx))

swidth = 5*sqrt(sigma2)
sbinsize = swidth/n_sbins;#sbins[2] - sbins[1]
base_sbins    = collect(-swidth:sbinsize:swidth)

ps       = exp(-base_sbins.^2/(2*sigma2))
ps       = ps/sum(ps);

sbin_length = length(base_sbins)
binN = length(bin_centers)

mu = 0.
# for j in 2:binN

j = 20
if lam == 0
    mu = bin_centers[j]*exp(lam*dt)
else
    mu = (bin_centers[j] + c/lam)*exp(lam*dt) - c/lam
end
sbins = base_sbins + mu
plot(sbins, ps)
ax = gca()
ax[:set_ylim]([0,0.5])
# end           

In [ ]:
length(bin_centers)

In [ ]:
b = 2

In [ ]:
Dict([("a",a),("b",b)])

In [ ]:
tic()
d = 3
t1 = toc()

In [ ]:
matwrite("matfile2.mat",Dict([("d",d),("b",b),("c",c)]))

In [ ]:
file = matopen("matfile.mat")
if exists(file, "variable")
    write(file,"a",a,"b",b)
end
close(file)

In [ ]:
vars = matread("matfile2.mat")

In [ ]:
nprocs()

In [ ]:
ratname = "B069"
saveto_filename = *("julia_out_",ratname,".mat")

In [ ]:
rand()

In [ ]:
zeros(1,3)

In [ ]:
a = [3; 3; 2]

In [ ]:
sum(a)

In [ ]:
#######################################################
function version_one(mu,X)
  aux = 0.0
  for nn=1:size(X,2)
    aux +=  sum( (X[:,nn]-mu).^2)
  end

  return aux

end

#######################################################
function version_two(mu,X)
  N = size(X,2)
  println(eltype(mu))
  aux = zeros(eltype(mu),N)
  for nn=1:N
    aux[nn] = sum( (X[:,nn]-mu).^2)
  end
  return sum(aux)

end

In [ ]:
X = randn(2,3) # some random data
mu = [1.0;2.0] # arbitrary mean
@show version_one(mu,X) # works fine for me
@show version_two(mu,X) # works fine for me

In [ ]:
ForwardDiff.gradient( ARG -> version_one(ARG,X), mu)


In [ ]:
ForwardDiff.gradient( ARG -> version_two(ARG,X), mu)